In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [85]:
import os
import pandas as pd
import glob
import pickle

os.sys.path.insert(0, '../evaluation')

from evaluate import evaluate_all_systems, preprocess_all_models

In [101]:
models = [os.path.basename(p) for p in glob.glob(f'../data/models/dev/*')]

dfs = []

for model in models:
    if os.path.isfile(f'../data/models/dev/{model}/system_evaluation.csv'):
        df_ = pd.read_csv(f'../data/models/dev/{model}/system_evaluation.csv', index_col=['subset', 'references', 'metric'])
        dfs.append(df_)

scores_df = pd.concat(dfs, keys=models).reset_index(level=[1, 2], drop=True).unstack()
scores_df.columns = scores_df.columns.droplevel()

In [102]:
params_dfs = []
for model in models:
    with open(f'../data/models/dev/{model}/params.pkl', 'rb') as f:
        params_dfs.append(pd.DataFrame([pickle.load(f)], index=[model]))
    

params_df = pd.concat(params_dfs)

In [103]:
df = pd.merge(scores_df, params_df, left_index=True, right_index=True)

cols = ['bleu', 'meteor', 'ter', 'dp_scorer', 'max_dp', 'sa_scorer', 'max_sa', 'tems_lm_name', 'tems_lm_n', 'max_tems', 'referrer', 'txs_lm_name', 'txs_lm_n']
df.sort_values('bleu', ascending=False).loc[:, cols]

,bleu,meteor,ter,dp_scorer,max_dp,sa_scorer,max_sa,tems_lm_name,tems_lm_n,max_tems,referrer,txs_lm_name,txs_lm_n
5530564602710747621,57.15,0.432661,0.420168,ltr_lasso,2,ltr_lasso,4,random,3,2,counter,random,3
-519698929690133953,56.97,0.435921,0.421631,ltr_lasso,2,ltr_lasso,4,lower,6,2,counter,lower,6
-5320594571956842214,56.01,0.430705,0.429773,ltr_lasso,2,ltr_lasso,4,lower,3,2,counter,lower,6
-3366481570762348342,55.82,0.433996,0.434258,ltr_lasso,2,ltr_lasso,4,lower,6,2,counter,random,6
710608634447711577,51.74,0.425727,0.465169,ltr_lasso,2,ltr_lasso,4,lower,3,2,counter,lower,3
-2991743951804606957,51.31,0.428404,0.472775,ltr_lasso,2,ltr_lasso,4,random,3,2,counter,lower,6
3090075189229807599,49.19,0.418883,0.510023,ltr_lasso,2,ltr_lasso,4,lower,3,2,counter,random,6
-8839139221232389730,48.84,0.417725,0.516703,ltr_lasso,2,ltr_lasso,4,random,3,2,counter,random,6
-6048291712712942420,47.47,0.413125,0.530159,ltr_lasso,2,ltr_lasso,4,lower,6,2,counter,lower,3
3993086837664471556,40.46,0.403896,0.623085,ltr_lasso,2,ltr_lasso,4,lower,3,2,counter,random,3


In [97]:
df.dp_scorer.value_counts()

ltr_lasso    64
random       64
Name: dp_scorer, dtype: int64